# Spark Development Notebook 


add loads of details here ... 

In [3]:
# imports 
import os 
import pandas as pd
from pyspark.sql import SparkSession

In [4]:
# ensure SPARK_HOME exists 
print(os.getenv('SPARK_HOME'))

/opt/spark-3.3.4-bin-hadoop3


In [ ]:
# Set up the Spark session
spark = (SparkSession.builder 
    .appName("Spark Dev App") 
    .master("spark://spark-master:7077")   # Connect to the Spark cluster
    # .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")  # MinIO endpoint
    # .config("spark.hadoop.fs.s3a.access.key", "minio_admin")   # MinIO access key
    # .config("spark.hadoop.fs.s3a.secret.key", "fnSZspu4tOETPLQ")   # MinIO secret key
    # .config("spark.hadoop.fs.s3a.path.style.access", "true")  # Required for MinIO
    # .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")   # S3A connector
    # .config("spark.sql.catalog.my_catalog", "org.apache.iceberg.spark.SparkCatalog")   # Iceberg catalog configuration
    # .config("spark.sql.catalog.my_catalog.type", "hadoop")   # Iceberg catalog type
    # .config("spark.sql.catalog.my_catalog.warehouse", "s3a://my-bucket/iceberg")   # Iceberg warehouse path in MinIO
    # .config("spark.jars", "$SPARK_HOME/jars/iceberg-spark-runtime-3.3_2.12-1.2.0.jar")   # Load Iceberg runtime JAR
    # .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")   # Disable SSL (if MinIO is running without SSL)
    .getOrCreate()
)

# Confirm the Spark session is up and running
spark